# 11. Interfaces: From Protocols to ABCs

## 11.1 파이썬에서의 인터페이스와 프로토콜

**프로토콜** → 파이썬과 같은 동적 자료형을 제공하는 언어에서 다형성을 제공하는 비공식 인터페이스

**인터페이스** → 클래스가 상속하거나 구현한 공개 속성(메서드나 데이터 속성)들의 집합.  
보충하자면, 시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드의 일부

프로토콜은 상속과 무관하며, 인터페이스지만 비공식적이다. 즉 문서와 관례에 따라 정의되지만, 공식 인터페이스처럼 강제할 수 없다.

## 11.2 파이썬은 시퀀스를 찾아낸다

In [2]:
class Foo :
    def __getitem__(self, pos) :
        return range(0, 30, 10)[pos]

f = Foo()
print(f[1])
for i in f : print(i)

10
0
10
20


위의 Foo 클래스는 abc.Sequence를 상속하지 않으며, 시퀀스 프로토콜 메서드 중 \_\_getitem\_\_() 메서드 하나만 구현한다.  
\_\_iter\_\_() 메서드는 구현되어 있지 않지만, 대체 수단인 \_\_getitem\_\_() 메서드가 구현되어 있으므로 Foo 객체를 반복할 수 있다.

시퀀스 프로토콜의 중요성을 보여주는 예시라 할 수 있겠다.

## 11.3 표준 라이브러리의 ABC

파이썬 2.6 이후 표준 라이브러리에 ABC가 포함되었다. 대부분의 ABC 패키지는 collectoins.abc 모듈에 정의되어 있고, 이 모듈에 정의된 ABC들이 가장 많이 사용된다.

대표적인 클래스로는 Iterable, Container, Sized, Sequence, Mapping, Set, MappingView, Callable, Hashable, Iterator 등이 있다.

# 12. Built-in Types Inheritance and Multiple Inheritance

## 12.1 내장 자료형의 상속

파이썬 2.2 이전까지는 list나 dict 등 내장 자료형을 상속할 수 없었다. 2.2 이후부터는 내장 자료형을 상속할 수 있게 되었지만, C언어로 작성된 내장 클래스의 코드는 사용자가 오버라이드한 코드를 호출하지 않으므로 각별한 주의가 필요하다.

공식적으로 CPythonㅇ느 내장 자료형의 서브클래스에서 오버라이드한 메서드가 언제 호출되는지, 혹은 호출되지 않는지에 대해 명확한 규칙을 정의하지 않는다. 일반적으로 서브클래스에서 오버라이드한 메서드는 같은 객체의 다른 내장 메서드에 의해 결코 호출되지 않는다.

아래 예시 코드를 통해 이 문제를 확인할 수 있다.

In [5]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1)
print(dd)
dd['two'] = 2
print(dd)
dd.update(three=3)
print(dd)

{'one': 1}
{'one': 1, 'two': [2, 2]}
{'one': 1, 'two': [2, 2], 'three': 3}


[] 연산자는 오버라이드한 \_\_setitem\_\_()을 호출하므로 'two'가 복제된 [2, 2]에 매핑되지만, \_\_setitem\_\_()를 호출하지 않는 update() 메서드를 이용한 'three'는 그렇지 않다.

내장 자료형은 '슈퍼클래스에서 구현된 메서드 안에서 호출하더라도 메서드 검색은 대상 객체(self)의 클래스에서 시작해야 한다'는 객체지향 프로그래밍의 기본 규칙을 어기고 있다. (단, \_\_missing\_\_() 메서드는 예외적인 상황을 처리하는 메서드이므로 문서화된 대로 작동한다.)

그리고 이는, collections.UserDict를 상속하는 방식으로 해결할 수 있다.

## 10.2 다중 상속

다중 상속을 지원하는 언어에서는 별개의 상위 클래스가 동일한 이름으로 메서드를 구현할 때 발생하는 이름 충돌 문제를 해결해야 한다. 이를 **다이아몬드 문제**라고 한다.

In [6]:
class A:
    def ping(self):
        print('ping:', self)

class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [8]:
d = D()
print(d.pong())   #1
print(C.pong(d))  #2

pong: <__main__.D object at 0x0000020F440270F0>
None
PONG: <__main__.D object at 0x0000020F440270F0>
None


B 클래스와 C 클래스 모두 pong() 메서드를 구현하고 있지만, C.pong() 메서드는 대문자로 출력한다.

이런 상황에서, D 객체에서 d.pong()을 호출하면 실제 어느 pong() 메서드가 호출될 것인가? 실제로 실험을 해 보면, B 클래스의 메서드가 호출됨을 알 수 있다.(#1)

이러한 모호성은 클래스명을 직접 명시함으로써 해결할 수 있다. (#2)

파이썬이 상속 그래프를 조회할 때는 특정한 순서를 따르므로, d.pong()과 같은 호출은 사실 동작이 정해져 있는 호출이다. 이 조회 순서를 **메서드 결정 순서(Method Resolution Order)**라고 하며, 이는 \_\_mro\_\_ 속성을 이용해 확인할 수 있다.

In [9]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

다중 상속의 필요성에 대해 의문을 가질 수도 있지만, 다중 상속이 잘못된 것은 아니다. 파이썬 표준 라이브러리에서는 collections.abs 패키지에서 다중 상속을 많이 사용하지만 이것이 논란거리가 되지는 않는다. 다중 상속을 올바르게 다루는지 알아보기 위해서는, 다음의 8가지 원칙을 준수하는지 확인해 보는 것이 좋다.

1. 인터페이스 상속과 구현 상속을 구분한다.
2. ABC를 이용해서 인터페이스를 명확히 한다.
3. 코드를 재사용하기 위해 믹스인을 사용한다.
4. 이름을 통해 믹스인임을 명확히 한다.
5. ABC가 믹스인이 될 수는 있지만, 믹스인이라고 해서 ABC인 것은 아니다.
6. 두 개 이상의 구상 클래스에서 상속받지 않는다.
7. 사용자에게 집합 클래스를 제공한다.
8. 클래스 상속보다 객체 구성을 사용한다.